In [ ]:
import pandas as pd
from models import base_res_net, small_res_net, efficient_net
import tensorflow as tf
import tensorflow.compat.v1 as tfc

from src import InputPipeline

%load_ext autoreload
%autoreload 2

## Check GPU for tf

In [ ]:
# Some GPU setup
# for documentation about using gpus refer to: https://www.tensorflow.org/install/pip#windows-wsl2

tf.keras.backend.clear_session()

device_name = tf.test.gpu_device_name()
if not device_name:
  raise SystemError('GPU device not found')
try:  # prevent a previous session from being alive
  sess.close() 
except:
  pass

tfc.enable_eager_execution()
gpu_options= tfc.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tfc.InteractiveSession(config=tfc.ConfigProto(gpu_options=gpu_options))

### Optional Stuff

In [ ]:
train_df = pd.read_csv("data/train_images.csv")
class_weights = train_df["label"].value_counts()

In [ ]:
class_weights = class_weights / sum(class_weights)
class_weights = {k-1: v for k,v in class_weights.to_dict().items()}

## Configure Parameters

In [ ]:
INPUT_SHAPE = (100,100,3)

CONF = {
  "learning_rate": 0.00001,
  "batch_size": 16,
  "epochs": 50,
  "loss_function": "sparse_categorical_crossentropy",
  "metric": "accuracy",
  "decay": 0.00004
}

## Make Input Pipelines

In [ ]:
# Input pipeline for subspecies
sub_species_input_pipeline = InputPipeline(splits=(0.8,0.0,0.2), channels=3, batch_size=CONF["batch_size"], size=INPUT_SHAPE[:2])
sub_species_input_pipeline.make_train_datasets(directory="data/train_images/train_images")

# Input pipeline for species
species_input_pipeline = InputPipeline(splits=(0.8,0.0,0.2), channels=3, batch_size=CONF["batch_size"], size=INPUT_SHAPE[:2])
species_input_pipeline.make_train_datasets(directory="data/train_images/species_classify")

In [ ]:
sub_species_cached_train, sub_species_cached_val = sub_species_input_pipeline.get_cached_train_datasets()
species_cached_train, species_cached_val = species_input_pipeline.get_cached_train_datasets()

## Training a Model

In [ ]:
from modeling import train_classifier

In [ ]:
# making the subspecies classifier
train_classifier(
  model_name="subspecies_classifier",
  input_shape=INPUT_SHAPE,
  classes_to_classify=200,
  configuration=CONF,
  model=base_res_net, 
  train_dataset=sub_species_cached_train,
  validation_dataset=sub_species_cached_val,
  class_weights=class_weights,
)

In [ ]:
# making the species classifier
train_classifier(
  model_name="species_classifier",
  input_shape=INPUT_SHAPE,
  classes_to_classify=70,
  configuration=CONF,
  model=efficient_net, 
  train_dataset=species_cached_train,
  validation_dataset=species_cached_val,
)

## Predict Stuff

In [ ]:
from modeling import predict
import pickle

In [ ]:
with open("mapping.pickle", "rb+") as f:
  mapping = pickle.load(f)

In [ ]:
sub_species_input_pipeline.make_test_dataset(directory="data/test_images/test_images")
test_cached = sub_species_input_pipeline.get_cached_test_datasets()

In [ ]:
predict(
  species_classifier="species_classifier",
  subspecies_classifier="subspecies_classifier",
  dataset=test_cached,
  species_subspecies_dict=mapping
)